In [103]:
from trading_ig.rest import IGService
import py_vollib.black_scholes_merton.implied_volatility as iv
import py_vollib_vectorized
import quantutils.dataset.pipeline as ppl
import pandas as pd

ig_service = IGService("rufusbets", "wilkoBETS@!(79", "11051b11fa833b5f6eab76e5e96baa9664cef439", "DEMO", "Z4HC2M")

In [104]:
ig = ig_service.create_session(version='3')
ig

{'clientId': '103088488',
 'accountId': 'Z4HC2M',
 'timezoneOffset': 0,
 'lightstreamerEndpoint': 'https://demo-apd.marketdatasystems.com',
 'oauthToken': {'access_token': '139c84d4-15bf-430d-aaec-c3724e6ec350',
  'refresh_token': '54b83ffb-3660-4961-a36d-e4b91b807cc0',
  'scope': 'profile',
  'token_type': 'Bearer',
  'expires_in': '60'}}

In [106]:
result = ig_service.fetch_historical_prices_by_epic_and_date_range(epic='OP.D.OIL1.5000C.IP', start_date="2022-09-07 19:00:00", end_date="2022-09-12 00:00:00")
result

{'prices':                         bid                             ask                  \
                        Open    High     Low   Close    Open    High     Low   
 DateTime                                                                      
 2022-09-07 19:00:00  3201.8  3219.1  3143.0  3159.8  3213.8  3231.1  3155.0   
 2022-09-08 13:00:00  3191.8  3255.7  3156.1  3222.7  3203.8  3267.7  3168.1   
 2022-09-08 14:00:00  3222.2  3340.9  3222.2  3332.0  3234.2  3352.9  3234.2   
 2022-09-08 15:00:00  3335.9  3335.9  3235.5  3253.9  3347.9  3347.9  3247.5   
 2022-09-08 16:00:00  3206.8  3341.7  3172.6  3300.5  3218.8  3353.7  3184.6   
 2022-09-08 17:00:00  3305.4  3334.2  3253.9  3301.0  3317.4  3346.2  3268.3   
 2022-09-08 18:00:00  3304.9  3367.0  3303.4  3325.6  3316.9  3383.8  3315.4   
 2022-09-08 19:00:00  3326.6  3368.3  3267.6  3273.6  3338.6  3380.3  3282.7   
 2022-09-09 13:00:00  3438.5  3532.5  3430.1  3529.0  3450.5  3544.5  3442.1   
 2022-09-09 14:00:00  3536.5  

In [109]:
ig_service.fetch_historical_prices_by_epic_and_date_range(epic='OP.D.OIL1.5000C.IP', resolution="D", start_date="2022-09-07 19:00:00", end_date="2022-09-12 00:00:00")

{'prices':                bid                             ask                        
               Open    High     Low   Close    Open    High     Low   Close
 DateTime                                                                  
 2022-09-08  3155.1  3368.3  3089.8  3248.7  3169.1  3383.8  3101.8  3262.7
 2022-09-09  3248.2  3663.5  3236.3  3574.9  3262.2  3675.5  3250.3  3586.9,
 'instrumentType': 'OPT_COMMODITIES',
 'allowance': {'remainingAllowance': 9911,
  'totalAllowance': 10000,
  'allowanceExpiry': 548095}}

In [102]:
epic = ig_service.fetch_market_by_epic("EN.D.CL.FWP2.IP").snapshot
price = epic.bid + epic.offer / 2.0
price

12846.0

In [111]:
optionData = ig_service.fetch_sub_nodes_by_node("342136543")["nodes"]
optionData


,id,name
0,342136643,10001-10500
1,342136652,10501-11000
2,342136610,11001-11500
3,342136613,11501-12000
4,342136628,12001-12500
5,342136688,12501-13000
6,342136757,13001-13500
7,342136673,13501-14000
8,342136619,14001-14500
9,342136640,14501-15000


In [112]:
#optionId = option["id"].values[0]
optionStrikes = ig_service.fetch_sub_nodes_by_node("342136859")["markets"]
optionStrikes

,delayTime,epic,netChange,lotSize,expiry,instrumentType,instrumentName,high,low,percentageChange,updateTime,updateTimeUTC,bid,offer,otcTradeable,streamingPricesAvailable,marketStatus,scalingFactor
0,0,OP.D.OIL1.5000C.IP,8564.0,10,NOV-22,OPT_COMMODITIES,US Crude 5000 CALL,3675.5,3300.0,-0.78,30652000,08:30:52,3559.4,3573.4,True,True,EDITS_ONLY,1
1,0,OP.D.OIL1.5000P.IP,8564.0,10,NOV-22,OPT_COMMODITIES,US Crude 5000 PUT,10.0,0.0,-0.78,72911000,20:15:11,0.0,10.0,True,True,EDITS_ONLY,1


In [108]:
df = pd.DataFrame()

for _,strikeRange in optionStrikes.iterrows():
    optionDetails = ig_service.fetch_sub_nodes_by_node(strikeRange["id"])["markets"]
    optionDetails = optionDetails.set_index("epic")
    df = ppl.merge(df, optionDetails)
    
df["type"] = [x.split()[-1] for x in df["instrumentName"].values]
df["strike"] = [int(x.split()[-2]) for x in df["instrumentName"].values]
df = df.sort_values("strike")

exp = ig_service.fetch_market_by_epic(df.index.values[0]).instrument.expiryDetails.lastDealingDate
df["expiry"] = parser.parse(exp)

df

,delayTime,netChange,lotSize,expiry,instrumentType,instrumentName,high,low,percentageChange,updateTime,updateTimeUTC,bid,offer,otcTradeable,streamingPricesAvailable,marketStatus,scalingFactor,type,strike
epic,,,,,,,,,,,,,,,,,,,
OP.D.OIL1.10000C.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 10000 CALL,239.8,141.6,-0.12,74647000,20:44:07,151.6,163.6,True,True,TRADEABLE,1.0,CALL,10000
OP.D.OIL1.10000P.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 10000 PUT,1587.8,1316.8,-0.12,74647000,20:44:07,1509.1,1521.1,True,True,TRADEABLE,1.0,PUT,10000
OP.D.OIL1.10050C.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 10050 CALL,229.3,134.1,-0.12,74647000,20:44:07,143.7,155.7,True,True,TRADEABLE,1.0,CALL,10050
OP.D.OIL1.10050P.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 10050 PUT,1629.8,1356.3,-0.12,74647000,20:44:07,1551.2,1563.2,True,True,TRADEABLE,1.0,PUT,10050
OP.D.OIL1.10100C.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 10100 CALL,219.2,126.9,-0.12,74647000,20:44:07,136.1,148.1,True,True,TRADEABLE,1.0,CALL,10100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OP.D.OIL1.9850P.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 9850 PUT,1463.8,1200.9,-0.12,74647000,20:44:07,1385.2,1397.2,True,True,TRADEABLE,1.0,PUT,9850
OP.D.OIL1.9900C.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 9900 CALL,262.1,157.7,-0.12,74647000,20:44:07,168.6,180.6,True,True,TRADEABLE,1.0,CALL,9900
OP.D.OIL1.9900P.IP,0.0,8642.5,10.0,NOV-22,OPT_COMMODITIES,US Crude 9900 PUT,1504.8,1239.1,-0.12,74647000,20:44:07,1426.1,1438.1,True,True,TRADEABLE,1.0,PUT,9900


In [102]:
exp = ig_service.fetch_market_by_epic(df.index.values[0]).instrument.expiryDetails.lastDealingDate
df["expiry"] = parser.parse(exp)

In [117]:
from dateutil import parser

In [77]:
vols = iv.implied_volatility(result["prices"]["ask"]["Close"].values, 9136.5, 8500, 0.093, 0.0, 'c', q=0)

result["prices"]["iv"] = vols.values
result["prices"]

bid                         ask                       \
                      Open   High    Low  Close   Open   High    Low  Close   
DateTime                                                                      
2022-09-06 19:00:00  622.6  671.1  622.0  655.9  634.6  683.1  634.0  667.9   

                           iv  
                               
DateTime                       
2022-09-06 19:00:00  0.200467

In [5]:
vols = iv.implied_volatility(result["prices"]["ask"]["Close"].values, 9136.5, 8500, 0.093, 0.0, 'c', q=0)

result["prices"]["iv"] = vols.values
result["prices"]

bid                             ask                  \
                       Open    High     Low   Close    Open    High     Low   
DateTime                                                                      
2022-06-01 13:00:00  2832.4  2948.3  2827.7  2902.4  2842.4  2958.3  2837.7   
2022-06-01 14:00:00  2922.7  3043.5  2904.8  2991.2  2932.7  3053.5  2914.8   
2022-06-01 15:00:00  2997.4  3005.4  2881.5  2902.7  3007.4  3015.4  2891.5   
2022-06-01 16:00:00  2901.7  2957.9  2882.4  2915.3  2911.7  2967.9  2892.4   
2022-06-01 17:00:00  2915.8  2965.2  2867.2  2963.8  2925.8  2975.2  2877.2   
...                     ...     ...     ...     ...     ...     ...     ...   
2022-08-11 15:00:00  1041.5  1054.0   993.6  1049.9  1053.5  1066.0  1005.6   
2022-08-11 16:00:00  1051.3  1088.0  1046.3  1078.1  1063.3  1100.0  1058.3   
2022-08-11 17:00:00  1079.1  1151.2  1079.1  1122.2  1091.1  1163.2  1091.1   
2022-08-11 18:00:00  1124.3  1156.4  1111.8  1150.6  1136.3  1168.4  1123.8   
2022-08-11 19:00:00  1152.4  1161.4  1098.8  1110.5  1164.4  1173.4  1110.8   

                                   iv  
                      Close            
DateTime                               
2022-06-01 13:00:00  2912.4  2.464216  
2022-06-01 14:00:00  3001.2  2.553728  
2022-06-01 15:00:00  2912.7  2.464518  
2022-06-01 16:00:00  2925.3  2.477192  
2022-06-01 17:00:00  2973.8  2.526061  
...                     ...       ...  
2022-08-11 15:00:00  1061.9  0.651930  
2022-08-11 16:00:00  1090.1  0.680095  
2022-08-11 17:00:00  1134.2  0.723888  
2022-08-11 18:00:00  1162.6  0.751954  
2022-08-11 19:00:00  1122.5  0.712296  

[350 rows x 9 columns]

In [60]:
from datetime import date, timedelta

today = date.today()
tomorrow = today + timedelta(days=1)

In [78]:
optionStrikes["Date_Time"] = today
optionStrikes["type"] = [x.split()[-1] for x in optionStrikes["instrumentName"].values]
x = optionStrikes.set_index(["Date_Time", "epic"])

In [85]:
ppl.merge(pd.DataFrame(index = ["Date_Time", "epic"]), x)

ValueError: cannot join with no overlapping index names

In [96]:
datetime.now().days

AttributeError: 'datetime.datetime' object has no attribute 'days'

In [92]:
ppl.merge(pd.DataFrame(index=idx), x)

ValueError: cannot join with no overlapping index names